In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

def load_checkpoint_for_training(model, optimizer, scheduler, checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    
    # Load model weights
    model.load_state_dict(checkpoint['model_state_dict'])
    
    # Load optimizer state
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    # Load scheduler state
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    # Get the epoch and loss
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    
    print(f"Loaded checkpoint from epoch {epoch} with loss: {loss:.4f}")
    
    return model, optimizer, scheduler, epoch, loss

def load_model_for_inference(model, model_path):
    # Load the model state
    model_state = torch.load(model_path, map_location='cpu')
    
    # Load into the model
    model.load_state_dict(model_state)
    
    # Set to evaluation mode
    model.eval()
    
    print(f"Loaded model from {model_path} for inference")
    
    return model

In [ ]:
def generate_text(model, tokenizer, input_text, max_length=50):
    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    
    # Move to the same device as model
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Generate
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length,
        num_beams=4,
        early_stopping=True
    )
    
    # Decode
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return generated_text

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# Load the best model weights
model = load_model_for_inference(model, "t5_finetuned.pt_best.pt")

# Move to device for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Generate text
input_text = "translate English to German: How are you?"
generated_text = generate_text(model, tokenizer, input_text)
print(f"Input: {input_text}")
print(f"Output: {generated_text}")